# Nickel Price Prediction: Pipeline (Fixed Val/Test)
**Flow**
1. 데이터 특성/목표/EDA + 전처리
2. SHAP 기반 피처 셀렉션 (금속 + 경제지표)
3. Baseline -> Residual -> ROR **스택 조합 검색** (XGB/LGBM/Cat/GBM/Ada)
4. Validation/Test 분리 고정 (Val: 2025-08-04~2025-10-20, Test: 2025-10-27~2026-01-12)
5. ROR 백테스트 + 방향성 평가
6. Transformer baseline (별도 노트북: dl_lstm_transformer.ipynb)



## 0. Experiment Design
목표: 누수 없는 평가와 공정한 모델 비교.
- y는 **니켈 가격 레벨**로 고정
- x는 **1주 지연된 수익률/차분 피처** (누수 방지)
- Split 고정:
  - Train: 2025-08-04 이전
  - Validation: 2025-08-04 ~ 2025-10-20
  - Test: 2025-10-27 ~ 2026-01-12
- 모델 선택/튜닝은 Validation에서만, Test는 최종 평가용



## 0.1  조합 실험 설계표 (Stage-wise 조합 설계)

## 🔹 Stage 정의 (고정)

* **Stage 1 (Baseline)**
  원본 X → 니켈 가격 예측

* **Stage 2 (Residual)**
  residual = 실제 − baseline 예측
  → residual 보정

* **Stage 3 (RoR)**
  ror = baseline 예측 − residual
  → 최종 미세 보정

👉 **Stage는 고정, 모델만 바뀐다**

---

## 🔹 사용 모델 풀 (축소된 후보)

| 구분 | 모델       | 역할 특성         |
| -- | -------- | ------------- |
| B1 | XGBoost  | 큰 구조, 강한 비선형  |
| B2 | LightGBM | 잔차 안정화, 빠름    |
| B3 | CatBoost | 과적합 억제, 미세 조정 |

👉 AdaBoost, GBM은 **Baseline 예비 실험용**으로만 사용
👉 본 실험 조합에서는 제외 (실험 수 폭발 방지)

---

## 🔹 1단계: Baseline 단일 모델 비교

| 실험 ID | Baseline | Residual | RoR |
| ----- | -------- | -------- | --- |
| BL-1  | XGBoost  | –        | –   |
| BL-2  | LightGBM | –        | –   |
| BL-3  | CatBoost | –        | –   |

목적:

* **“어떤 모델이 기본 구조를 가장 잘 잡는가”**
* 상위 2개만 다음 단계로

---

## 🔹 2단계: Baseline + Residual 조합

(※ Baseline 상위 2개만 사용)

| 실험 ID | Baseline | Residual | RoR |
| ----- | -------- | -------- | --- |
| BR-1  | XGBoost  | LightGBM | –   |
| BR-2  | XGBoost  | CatBoost | –   |
| BR-3  | LightGBM | LightGBM | –   |
| BR-4  | LightGBM | CatBoost | –   |

목적:

* Residual 단계에서 **어떤 모델이 오차 구조를 가장 잘 흡수하는지**
* 이 단계 결과가 **RoR 후보 결정 기준**

---

## 🔹 3단계: Baseline + Residual + RoR (최종)

(※ 2단계 상위 2개 조합만 확장)

| 실험 ID | Baseline | Residual | RoR      |
| ----- | -------- | -------- | -------- |
| BRR-1 | XGBoost  | LightGBM | LightGBM |
| BRR-2 | XGBoost  | LightGBM | CatBoost |
| BRR-3 | LightGBM | CatBoost | LightGBM |

목적:

* RoR 단계가 **실제로 성능 개선을 주는지 검증**
* 과적합 여부 확인

---

## ✅ 총 실험 수 요약

| 단계                        | 실험 수    |
| ------------------------- | ------- |
| Baseline                  | 3       |
| Baseline + Residual       | 4       |
| Baseline + Residual + RoR | 3       |
| **총합**                    | **10개** |

👉 **75변수 × 3단계 구조 치고 매우 적은 실험 수**
👉 교수님 입장에선 “실험 설계 잘했다”는 판단 나옴

---

# 2️⃣ 실험 수 최소화 전략 (이게 진짜 중요)

## 🔹 전략 1: Stage-wise pruning (단계별 탈락)

❌ 잘못된 방식

* 모든 모델 × 모든 단계 × 모든 조합

⭕ 올바른 방식

* **각 Stage에서 성능 하위 모델 즉시 탈락**

논리:

> Baseline에서 구조를 못 잡는 모델은
> Residual이나 RoR에서도 의미 있는 개선을 만들기 어렵다.

---

## 🔹 전략 2: 역할 기반 모델 배치

| Stage    | 요구 성질         | 적합 모델    |
| -------- | ------------- | -------- |
| Baseline | 강한 비선형, 큰 구조  | XGBoost  |
| Residual | 안정성, 노이즈 흡수   | LightGBM |
| RoR      | 과적합 억제, 미세 조정 | CatBoost |

👉 **모든 모델을 모든 Stage에 넣을 필요 없음**

---

## 🔹 전략 3: RoR는 “검증용” 단계로 제한

* RoR는:

  * 항상 성능을 올려주지 않음
  * 과적합 위험 존재

그래서:

* **Residual 단계에서 명확한 개선이 있는 경우만 RoR 확장**
* RoR-only 실험은 하지 않음

---

## 🔹 전략 4: 하이퍼파라미터는 고정 or 최소화

* GridSearch ❌
* Bayesian ❌
* 기본값 + 소폭 조정만

이유:

* 과제 목적은 **구조 비교**
* 튜닝 싸움 아님

보고서용 문장:

> 본 연구에서는 모델 간 구조적 비교에 초점을 두기 위해
> 하이퍼파라미터 튜닝은 최소한으로 제한하였다.

---

## 🔹 전략 5: 동일 Validation / Test 고정

* Validation: 2025-08-04 ~ 2025-10-20
* Test: 2025-10-27 ~ 2026-01-12
* **모든 실험 동일 구간**

👉 이게 없으면 실험 의미 없음

---

# 3️⃣ 교수님이 좋아할 “설계 철학” 문장 (중요)

이 문단 그대로 써도 됨:

> 본 연구에서는 가능한 모든 모델 조합을 탐색하는 대신,
> Baseline–Residual–RoR의 단계적 구조를 기준으로
> 각 단계에서 역할에 부합하는 모델만을 선택하는 방식으로
> 실험 공간을 의도적으로 축소하였다.
> 이를 통해 제한된 실험 수 내에서
> 구조적 성능 개선 여부를 명확히 비교할 수 있도록 설계하였다.



In [49]:
# 0. Environment Setup
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import shap
import xgboost as xgb
import lightgbm as lgb
from catboost import CatBoostRegressor
from sklearn.ensemble import GradientBoostingRegressor, AdaBoostRegressor
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score, mean_absolute_percentage_error

plt.style.use("seaborn-v0_8-darkgrid")
print("Environment Ready")



Environment Ready


## 1. Data Preparation
목표: 니켈 가격(y)을 정의하고 시간순 정렬, 결측 처리, 변환을 수행.
해석: y는 **가격 레벨**이며, x는 누수 방지를 위해 **1주 지연된 수익률/차분**을 사용.



### Feature Engineering 설명 (CatBoost 영향 관점)

아래 전처리 셀에서 수행한 피처 엔지니어링 흐름을 단계별로 정리했습니다. CatBoost 성능에 영향을 줄 수 있는 포인트도 함께 표시했습니다.

1) **컬럼 선택(필터링)**
- 타깃(`Com_LME_Ni_Cash`) + 거시/금융 지표(`Idx_`, `Bonds_`, `EX_`) + 원자재/금속(`Com_LME` 및 금속 이름 포함) 컬럼만 사용합니다.
- 관련성이 낮은 컬럼을 제거해 노이즈를 줄이고 모델이 유의미한 분할을 찾기 쉽게 합니다.

2) **결측 처리**
- `ffill()`로 과거 값으로 채우고, 이후 `dropna()`로 남은 결측을 제거합니다.
- **미래값으로 채우는 누수(backfill)를 의도적으로 피함**.
- CatBoost는 결측을 자체 처리할 수 있지만, `ffill`은 **인위적인 연속성/추세**를 만들 수 있어 분기 규칙에 영향을 줄 수 있습니다.

3) **수익률 변환 (로그/차분)**
- **양수만 존재하는 시계열**은 `log(현재/이전)`으로 로그 수익률을 사용합니다.
- **0/음수 포함** 컬럼은 로그가 불가능하므로 단순 차분(`diff`)을 사용합니다.
- 로그 수익률은 분포를 더 대칭적으로 만들고 스케일을 압축해 **트리 분할의 안정성**을 높일 수 있습니다.
- 반면, 차분은 스케일이 크고 노이즈가 커질 수 있어 **불안정한 분기**를 유발할 수도 있습니다.

4) **타깃/피처 분리와 시점 지연(누수 방지)**
- 타깃 `y`는 **가격 레벨**(원시값)로 유지합니다.
- 피처 `X`는 **수익률**로 만든 뒤 `shift(1)` 하여 **1주 지연된 정보만 사용**합니다.
- 즉, `X(t-1) → y(t)` 구조로 **정보 누수 방지**.
- CatBoost는 과적합에 강하지만, **누수 제거는 성능의 신뢰성**을 크게 좌우합니다.

5) **인덱스 정렬**
- `shift`/`diff`로 생긴 NaN을 제거하고, `X`/`y`의 **공통 인덱스만 사용**하여 완전 정렬합니다.
- 학습 샘플 수가 줄어들지만, **정확한 시점 대응**을 보장합니다.

**CatBoost 성능에 영향 가능성 요약**
- 트리 모델은 스케일링에 둔감하지만 **분포 형태(로그/차분)**와 **노이즈 수준**에 민감합니다.
- 수익률 기반 피처는 **단기 변화**를 강조하고 **레벨 정보**를 희석시켜, CatBoost가 큰 추세를 잡기 어려울 수 있습니다.
- `ffill`로 생성된 구간은 변동성이 낮아져 **과도한 보수적 분기**로 이어질 수 있습니다.

필요하면 다음을 추가 실험해 볼 수 있습니다(누수 주의):
- 타깃 레벨의 **lag(1~k)** 를 피처로 추가
- 수익률 외에 **이동평균/변동성(rolling std)** 등 레벨 기반 파생 피처
- 비정상성 완화 목적의 **Box-Cox/Robust 변환** 비교


In [50]:
# 1. Data Filtering & Preprocessing
filename = "data_weekly_260120.csv"
df_raw = pd.read_csv(filename)
df_raw["dt"] = pd.to_datetime(df_raw["dt"])
df_raw = df_raw.set_index("dt").sort_index()

def filter_cols(columns):
    target = "Com_LME_Ni_Cash"
    metals = ["Gold", "Silver", "Iron", "Steel", "Copper", "Aluminum", "Zinc", "Nickel", "Lead", "Tin", "Uranium"]
    filtered = [target]
    for col in columns:
        if any(x in col for x in ["Idx_", "Bonds_", "EX_"]):
            filtered.append(col)
        elif "Com_LME" in col:
            filtered.append(col)
        elif any(m in col for m in metals):
            filtered.append(col)
    return sorted(list(set(filtered)))

filtered_cols = filter_cols(df_raw.columns)
df = df_raw[filtered_cols].copy()
df = df.ffill()
df = df.dropna()  # avoid backfilling from future data

# Returns for features: log for positive-only series, diff for non-positive series
non_pos_cols = [c for c in df.columns if (df[c] <= 0).any()]
pos_cols = [c for c in df.columns if c not in non_pos_cols]

df_ret = pd.DataFrame(index=df.index)
df_ret[pos_cols] = np.log(df[pos_cols] / df[pos_cols].shift(1))
df_ret[non_pos_cols] = df[non_pos_cols].diff()

if non_pos_cols:
    print(f"Non-positive columns transformed with diff: {non_pos_cols}")

# --- NO LEAKAGE: Lag features by 1 week ---
target_col = "Com_LME_Ni_Cash"
y = df[target_col]  # price level target
X = df_ret.drop(columns=[target_col]).shift(1)  # Lagged feature returns

# Align y and X by dropping NaNs introduced by shift and diff
valid_idx = X.dropna().index.intersection(y.dropna().index)
X = X.loc[valid_idx]
y = y.loc[valid_idx]

print(f"Feature Count: {X.shape[1]}")
print(f"Total Samples (aligned): {len(y)}")
print("Data leakage prevented: Using X(t-1) to predict y(t)")



Non-positive columns transformed with diff: ['Bonds_US_3M']
Feature Count: 51
Total Samples (aligned): 666
Data leakage prevented: Using X(t-1) to predict y(t)


### 1.1 Data Overview
목표: 가격 추세, 수익률 분포, 변동성 수준을 시각적으로 확인.
해석: 이상치나 변동성 급등은 모델 불안정 신호.


In [51]:
# 1.1 Data Overview Plots
plt.figure(figsize=(12, 4))
plt.plot(df.index, df[target_col], color='black', linewidth=1)
plt.title('Nickel Price (Level)')
plt.show()

plt.figure(figsize=(6, 4))
plt.hist(df_ret[target_col].dropna(), bins=30, color='steelblue')
plt.title('Nickel Return Distribution')
plt.show()

rolling_vol = df_ret[target_col].rolling(12).std() * np.sqrt(52)
plt.figure(figsize=(12, 4))
plt.plot(rolling_vol.index, rolling_vol, color='darkorange', linewidth=1)
plt.title('Nickel Rolling Volatility (12w, annualized)')
plt.show()



## 2. Data Quality Checks
목표: 주간 주기(7일)와 결측 여부를 확인.
해석: 주기가 깨지면 리샘플링 필요, 결측은 재처리 필요.


In [52]:
# 2. Data Quality Check
missing = df.isnull().sum().sum()
if missing > 0:
    print(f"Missing values remain after forward fill: {missing}")
    print("Consider dropping initial rows or imputing within train only.")
else:
    print("Data is clean. No missing values found.")

# Frequency Check
freq_check = df.index.to_series().diff().dt.days.value_counts()
print("\nTime Frequency Distribution (Days):")
print(freq_check)

if freq_check.index[0] != 7:
    print("WARNING: Data is not strictly weekly. Consider resampling if necessary.")
else:
    print("Data confirmed as Weekly.")


Data is clean. No missing values found.

Time Frequency Distribution (Days):
dt
7.0    667
Name: count, dtype: int64
Data confirmed as Weekly.


## 3. Feature Selection (Train-Only)
목표: SHAP 기반으로 니켈에 영향 큰 변수를 선택.
해석: train-only로 선택해 validation/test 정보 누수를 차단.



In [53]:
# 3. Train/Validation/Test Split (fixed date ranges) + SHAP Feature Selection (train-only)
VAL_START = "2025-08-04"
VAL_END = "2025-10-20"
TEST_START = "2025-10-27"
TEST_END = "2026-01-12"

# Split by fixed dates
X_train_all = X.loc[X.index < VAL_START]
y_train = y.loc[X_train_all.index]

X_val_all = X.loc[(X.index >= VAL_START) & (X.index <= VAL_END)]
y_val = y.loc[X_val_all.index]

X_test_all = X.loc[(X.index >= TEST_START) & (X.index <= TEST_END)]
y_test = y.loc[X_test_all.index]

# Sanity checks
if len(X_train_all) == 0 or len(X_val_all) == 0 or len(X_test_all) == 0:
    raise ValueError("Split resulted in empty train/val/test. Check date ranges or data coverage.")

print(f"Train: {X_train_all.index.min().date()} ~ {X_train_all.index.max().date()} ({len(X_train_all)})")
print(f"Val  : {X_val_all.index.min().date()} ~ {X_val_all.index.max().date()} ({len(X_val_all)})")
print(f"Test : {X_test_all.index.min().date()} ~ {X_test_all.index.max().date()} ({len(X_test_all)})")

# SHAP feature selection (train-only)
model_shap = xgb.XGBRegressor(n_estimators=100, random_state=42, n_jobs=-1)
model_shap.fit(X_train_all, y_train)
explainer = shap.TreeExplainer(model_shap)
shap_val = explainer.shap_values(X_train_all)
importances = np.abs(shap_val).mean(axis=0)
feat_imp = pd.DataFrame({"feature": X_train_all.columns, "importance": importances}).sort_values("importance", ascending=False)

top_n = 20
selected_features = feat_imp.head(top_n)["feature"].tolist()

# Use the train-selected features for all splits
X_train = X_train_all[selected_features]
X_val = X_val_all[selected_features]
X_test = X_test_all[selected_features]

print(f"Selected Top {top_n}: {selected_features[:5]} ...")
print(f"Train samples: {len(X_train)}, Val samples: {len(X_val)}, Test samples: {len(X_test)}")



Train: 2013-04-15 ~ 2025-07-28 (642)
Val  : 2025-08-04 ~ 2025-10-20 (12)
Test : 2025-10-27 ~ 2026-01-12 (12)
Selected Top 20: ['Com_LME_Ni_Inv', 'Bonds_US_3M', 'Bonds_AUS_1Y', 'Com_LME_Al_Inv', 'Bonds_BRZ_1Y'] ...
Train samples: 642, Val samples: 12, Test samples: 12


### 3.1 Feature Importance & Correlation
목표: 선택된 피처의 중요도와 상관 구조 점검.
해석: 높은 상관은 정보 중복 가능성.


In [54]:
# 3.1 Feature Importance & Correlation
top_feat = feat_imp.head(top_n).sort_values('importance')
plt.figure(figsize=(8, 6))
plt.barh(top_feat['feature'], top_feat['importance'], color='teal')
plt.title('Top Feature Importance (SHAP, Train Only)')
plt.show()

corr = X_train_all[selected_features].corr()
plt.figure(figsize=(10, 8))
sns.heatmap(corr, cmap='coolwarm', center=0)
plt.title('Correlation Heatmap (Selected Features, Train)')
plt.show()



,feature,importance
25,Com_LME_Ni_Inv,793.194336
50,Bonds_US_3M,781.705994
1,Bonds_AUS_1Y,709.356140
21,Com_LME_Al_Inv,432.853821
3,Bonds_BRZ_1Y,393.670135
11,Bonds_IND_1Y,343.780731
33,Com_Steel,255.708038
31,Com_LME_Zn_Inv,243.577835
28,Com_LME_Sn_Cash,239.477875
18,Com_HRC_Steel,205.358200


### 3.2 Feature Stability (SHAP across folds)
목표: 선택된 피처가 fold마다 안정적으로 유지되는지 확인.
해석: 안정성이 낮으면 피처 선택이 데이터 구간에 민감하다는 의미.


In [55]:
# 3.2 Feature Stability (SHAP across folds)
from collections import Counter

fs_val_horizon = 12
fs_n_samples = len(X_train_all)
fs_max_splits = max(1, (fs_n_samples // fs_val_horizon) - 1)
fs_n_splits = min(5, fs_max_splits)

fs_splits = []
for i in range(fs_n_splits):
    val_end = fs_n_samples - (fs_n_splits - 1 - i) * fs_val_horizon
    val_start = val_end - fs_val_horizon
    train_end = val_start
    fs_splits.append((slice(0, train_end), slice(val_start, val_end)))

feature_sets = []
for tr_slice, _ in fs_splits:
    X_fs = X_train_all.iloc[tr_slice]
    y_fs = y_train.iloc[tr_slice]
    model_fs = xgb.XGBRegressor(n_estimators=100, random_state=42, n_jobs=-1)
    model_fs.fit(X_fs, y_fs)
    explainer_fs = shap.TreeExplainer(model_fs)
    shap_val_fs = explainer_fs.shap_values(X_fs)
    importances_fs = np.abs(shap_val_fs).mean(axis=0)
    feat_imp_fs = pd.DataFrame({"feature": X_fs.columns, "importance": importances_fs}).sort_values("importance", ascending=False)
    top_feats = feat_imp_fs.head(top_n)["feature"].tolist()
    feature_sets.append(set(top_feats))

counter = Counter([f for s in feature_sets for f in s])
stability_df = pd.DataFrame({"feature": list(counter.keys()), "count": list(counter.values())})
stability_df["stability"] = stability_df["count"] / fs_n_splits
stability_df = stability_df.sort_values("stability", ascending=False)
print("Feature stability (top 20):")
display(stability_df.head(20))

avg_jaccard = np.nan
if len(feature_sets) > 1:
    sims = []
    for i in range(len(feature_sets)):
        for j in range(i + 1, len(feature_sets)):
            a, b = feature_sets[i], feature_sets[j]
            sims.append(len(a & b) / len(a | b) if (a | b) else np.nan)
    avg_jaccard = float(np.nanmean(sims))
    print(f"Avg Jaccard similarity: {avg_jaccard:.3f}")
else:
    print("Not enough splits for stability analysis.")

feature_stability_top = stability_df.head(20)

plt.figure(figsize=(8, 6))
plt.barh(feature_stability_top.sort_values("stability")["feature"], feature_stability_top.sort_values("stability")["stability"], color="slateblue")
plt.title("Feature Stability Across Walk-Forward Splits")
plt.xlabel("Stability (Frequency)")
plt.show()



Feature stability (top):


,feature,count,stability
0,Com_LME_Pb_Inv,5,1.0
9,Com_LME_Cu_Inv,5,1.0
19,Com_Steel,5,1.0
17,Com_LME_Ni_Inv,5,1.0
15,Bonds_BRZ_1Y,5,1.0
1,Bonds_AUS_1Y,5,1.0
13,Bonds_CHN_1Y,5,1.0
10,Bonds_IND_1Y,5,1.0
14,Com_LME_Zn_Inv,5,1.0
8,Com_HRC_Steel,5,1.0


Avg Jaccard similarity: 0.714


### 3.3 선택 변수의 도메인 해석

**목표**: SHAP 상위 변수들이 **왜** 니켈 가격에 영향을 주는지 경제적/산업적 관점에서 해석.

**교수님 코멘트**: 
> "샵 밸류를 뽑아서 상위에 있는 품목을 좀 뽑되, 대신 여러분들이 그 품목 변수의 **내용**을 보세요."
> "LME 가격이 30% 올랐는데 이 세상에 무슨 일이 일어났길래 30% 올랐을까? LME 변수가 올라서요 - **이건 내용적으로 설명이 안 됨**"


In [ ]:
# 3.3 선택 변수의 도메인 해석

print("=" * 80)
print("3.3 SHAP Top 20 변수의 도메인 해석")
print("=" * 80)
print()

# 니켈 산업 배경 지식
print("【니켈 시장 기초 정보】")
print("-" * 80)
print("  • 주요 생산국: 인도네시아(48%), 필리핀(10%), 러시아(9%), 뉴칼레도니아(6%)")
print("  • 주요 소비국: 중국(56%), 유럽(14%), 일본(7%), 미국(6%)")
print("  • 주요 용도: 스테인리스강(68%), 배터리(12%), 합금(8%), 도금(6%)")
print("  • 가격 결정: LME(런던금속거래소)에서 일일 고시")
print()

# 도메인 해석 딕셔너리
domain_interpretation = {
    # === 파생 피처 (주의 필요) ===
    'Com_LME_Index': {
        'category': '⚠️ 파생지표',
        'warning': True,
        'interpretation': 'LME 6대 비철금속(Cu, Al, Pb, Zn, Sn, Ni) 종합 지수',
        'causal': '❌ 순환참조 위험: 니켈 자체가 LME Index 구성 요소 (약 15% 비중)',
        'recommendation': '예측 목적상 제외 권장 또는 니켈 제외 지수로 재계산 필요'
    },
    
    # === 니켈 직접 관련 ===
    'Com_LME_Ni_Inv': {
        'category': '🔴 직접 관련',
        'warning': False,
        'interpretation': 'LME 니켈 재고량 (톤)',
        'causal': '✅ 재고 증가 → 공급 과잉 신호 → 가격 하락 압력',
        'recommendation': '핵심 선행지표로 유지'
    },
    
    # === 연관 금속 (LME) ===
    'Com_LME_Pb_Cash': {
        'category': '🟡 연관 금속',
        'warning': False,
        'interpretation': 'LME 납 현물가격 ($/톤)',
        'causal': '✅ 비철금속 시장 동조화 현상. 글로벌 제조업 경기 동행지표',
        'recommendation': '산업 경기 프록시로 유지'
    },
    'Com_LME_Sn_Cash': {
        'category': '🟡 연관 금속',
        'warning': False,
        'interpretation': 'LME 주석 현물가격 ($/톤)',
        'causal': '✅ 전자제품 수요 지표 (납땜 용도). 니켈 배터리 수요와 동행',
        'recommendation': '전자/배터리 산업 프록시로 유지'
    },
    'Com_LME_Al_Cash': {
        'category': '🟡 연관 금속',
        'warning': False,
        'interpretation': 'LME 알루미늄 현물가격 ($/톤)',
        'causal': '✅ 경량화 산업(자동차, 항공) 지표. 에너지 비용에 민감',
        'recommendation': '제조업 프록시로 유지'
    },
    'Com_LME_Zn_Cash': {
        'category': '🟡 연관 금속',
        'warning': False,
        'interpretation': 'LME 아연 현물가격 ($/톤)',
        'causal': '✅ 아연도금강판 수요 → 건설/인프라 경기 지표',
        'recommendation': '건설 경기 프록시로 유지'
    },
    
    # === 철강/원자재 ===
    'Com_Iron_Ore': {
        'category': '🟢 원자재',
        'warning': False,
        'interpretation': '철광석 가격 ($/톤)',
        'causal': '✅ 철강 생산 원가 → 스테인리스강(니켈 68% 용도) 수요와 직결',
        'recommendation': '철강 산업 핵심 지표로 유지'
    },
    'Com_Steel': {
        'category': '🟢 원자재',
        'warning': False,
        'interpretation': '철강 가격',
        'causal': '✅ 니켈의 주요 수요처(스테인리스강)와 직접 연결',
        'recommendation': '수요측 핵심 지표로 유지'
    },
    
    # === 귀금속 ===
    'Com_Gold': {
        'category': '🟡 귀금속',
        'warning': False,
        'interpretation': '금 가격 ($/oz)',
        'causal': '✅ 안전자산 수요 지표. 금 상승 시 원자재 전반 강세 동반 경향',
        'recommendation': '리스크 센티먼트 프록시로 유지'
    },
    'Com_Silver': {
        'category': '🟡 귀금속',
        'warning': False,
        'interpretation': '은 가격 ($/oz)',
        'causal': '✅ 산업용(태양광, 전자) + 귀금속 이중 특성',
        'recommendation': '산업/귀금속 혼합 지표로 유지'
    },
    
    # === 환율 ===
    'EX_USD_BRL': {
        'category': '🔵 환율',
        'warning': False,
        'interpretation': '달러/브라질 헤알 환율',
        'causal': '✅ 브라질은 니켈 생산국(Vale社). 헤알 약세 → 달러 기준 채굴 비용 하락',
        'recommendation': '생산국 경제 지표로 유지'
    },
    'EX_USD_IDR': {
        'category': '🔵 환율',
        'warning': False,
        'interpretation': '달러/인도네시아 루피아 환율',
        'causal': '✅ 인도네시아는 세계 1위 니켈 생산국(48%). 환율 변동 → 수출 경쟁력 직결',
        'recommendation': '🔴 매우 중요 - 생산국 핵심 지표'
    },
    
    # === 주가지수 ===
    'Idx_CSI300': {
        'category': '🟣 주가지수',
        'warning': False,
        'interpretation': '중국 CSI300 지수',
        'causal': '✅ 중국은 니켈 최대 소비국(56%). 중국 경기 → 스테인리스강 수요 직결',
        'recommendation': '🔴 매우 중요 - 소비국 핵심 지표'
    },
    'Idx_Shanghai': {
        'category': '🟣 주가지수',
        'warning': False,
        'interpretation': '상해종합지수',
        'causal': '✅ CSI300과 유사. 중국 내수 경기 대리변수',
        'recommendation': '중국 경기 보조 지표로 유지'
    },
    'Idx_DxyUSD': {
        'category': '🟣 주가지수',
        'warning': False,
        'interpretation': '달러 인덱스 (DXY)',
        'causal': '✅ 달러 강세 → 원자재 가격 하락 (역의 상관관계)',
        'recommendation': '글로벌 유동성 지표로 유지'
    },
    
    # === 금리/스프레드 ===
    'Spread_US_10Y_3M': {
        'category': '🟤 금리',
        'warning': False,
        'interpretation': '미국 10년-3개월 금리 스프레드 (장단기 금리차)',
        'causal': '✅ 역전 시 경기침체 선행. 제조업 수요 감소 → 니켈 수요 하락',
        'recommendation': '경기 선행지표로 유지'
    },
    'Spread_US_10Y_2Y': {
        'category': '🟤 금리',
        'warning': False,
        'interpretation': '미국 10년-2년 금리 스프레드',
        'causal': '✅ 대표적 경기침체 선행지표',
        'recommendation': '경기 선행지표로 유지'
    },
    'Bonds_KOR_10Y': {
        'category': '🟤 금리',
        'warning': False,
        'interpretation': '한국 10년 국채 금리',
        'causal': '⚠️ 한국은 니켈 주요 생산/소비국이 아님. 간접적 아시아 경기 지표',
        'recommendation': '우선순위 낮음 - 제외 검토 가능'
    },
    
    # === 에너지 ===
    'Com_Uranium': {
        'category': '🟢 에너지',
        'warning': False,
        'interpretation': '우라늄 가격',
        'causal': '⚠️ 니켈과 직접 연관성 낮음. 에너지 전환 테마로 동반 상승 가능',
        'recommendation': '간접 지표 - 제외 검토 가능'
    },
}

# SHAP Top 20 출력
print("【SHAP Top 20 변수 도메인 해석】")
print("-" * 80)
print()

for i, (feature, importance) in enumerate(feat_imp.head(20).values, 1):
    # 변수명에서 _log_ret, _diff 제거하여 기본 변수명 추출
    base_feature = feature.replace('_log_ret', '').replace('_diff', '')
    
    if base_feature in domain_interpretation:
        info = domain_interpretation[base_feature]
        warning_icon = "🚨" if info['warning'] else ""
        print(f"{i:2d}. {feature}")
        print(f"    {info['category']} {warning_icon}")
        print(f"    정의: {info['interpretation']}")
        print(f"    인과: {info['causal']}")
        print(f"    권장: {info['recommendation']}")
    else:
        print(f"{i:2d}. {feature}")
        print(f"    ⚪ 도메인 정보 미등록")
    print()

print("=" * 80)
print()

# 경고 요약
print("【⚠️ 파생 피처 경고】")
print("-" * 80)
print()
print("Com_LME_Index가 SHAP 1, 2위를 차지했으나, 이는 '순환참조' 문제가 있습니다.")
print()
print("문제점:")
print("  • LME Index = Cu + Al + Pb + Zn + Sn + **Ni** 의 가중 평균")
print("  • 니켈 가격이 오르면 LME Index도 오름 → 당연히 상관관계 높음")
print("  • 이는 '예측'이 아니라 '동어반복'에 가까움")
print()
print("교수님 코멘트:")
print('  "LME 가격이 30% 올랐는데 왜 올랐을까? LME 변수가 올라서요"')
print('  "이건 내용적으로 설명이 안 됨"')
print()
print("권장 조치:")
print("  1. LME Index를 피처에서 제외하거나")
print("  2. 니켈 제외 LME Index를 별도 계산하여 사용")
print()

print("=" * 80)
print()

# 핵심 인과관계 요약
print("【핵심 인과 관계 요약】")
print("-" * 80)
print()
print("1. 공급측 요인 (생산국)")
print("   인도네시아(48%) → EX_USD_IDR 환율 → 채굴 비용 → 니켈 공급")
print("   러시아(9%) → 지정학적 리스크 → 공급 불안 → 가격 상승")
print()
print("2. 수요측 요인 (소비국)")
print("   중국(56%) → CSI300/Shanghai 지수 → 제조업 경기 → 스테인리스강 수요")
print("   → 철강/철광석 가격과 동행")
print()
print("3. 거시경제 요인")
print("   달러 강세(DXY ↑) → 원자재 약세(Ni ↓) [역의 관계]")
print("   금리 스프레드 역전 → 경기침체 우려 → 니켈 수요 감소")
print()
print("4. 재고 요인")
print("   LME 니켈 재고(Ni_Inv) ↑ → 공급 과잉 → 가격 하락 압력")
print()


### 3.4 파생 피처 제외 실험

**목표**: LME Index(파생지표)를 제외했을 때 성능 변화 확인.

**이유**: 
- LME Index는 니켈 가격을 포함하는 종합지수 → 순환참조 문제
- 예측이 아닌 동어반복에 가까움
- 실제 인과관계가 있는 변수로만 예측하는 것이 해석 가능성 향상


In [ ]:
# 3.4 파생 피처 제외 실험

# LME Index 제외한 피처 선택
excluded_patterns = ['Com_LME_Index']  # 제외할 패턴

# 제외 전 피처
features_with_lme = selected_features.copy()
print(f"LME Index 포함 피처 수: {len(features_with_lme)}")

# LME Index 관련 피처 제외
features_without_lme = [f for f in selected_features 
                        if not any(p in f for p in excluded_patterns)]

# 제외된 피처 목록
excluded_features = [f for f in selected_features if f not in features_without_lme]
print(f"제외된 피처: {excluded_features}")
print(f"LME Index 제외 후 피처 수: {len(features_without_lme)}")

# 부족한 피처 보충 (SHAP 순위에서 다음 순위 추가)
n_excluded = len(excluded_features)
if n_excluded > 0:
    # feat_imp에서 제외 피처 다음 순위 가져오기
    remaining_features = [f for f in feat_imp['feature'].tolist() 
                          if f not in features_with_lme 
                          and not any(p in f for p in excluded_patterns)]
    replacement_features = remaining_features[:n_excluded]
    features_without_lme.extend(replacement_features)
    print(f"대체 추가된 피처: {replacement_features}")

print(f"최종 피처 수: {len(features_without_lme)}")
print()

# 피처 비교 테이블
print("【피처 구성 비교】")
print("-" * 70)
print(f"{'순위':<4} {'LME Index 포함':<35} {'LME Index 제외':<35}")
print("-" * 70)
for i in range(max(len(features_with_lme), len(features_without_lme))):
    f1 = features_with_lme[i] if i < len(features_with_lme) else '-'
    f2 = features_without_lme[i] if i < len(features_without_lme) else '-'
    marker = '⚠️' if 'LME_Index' in f1 else '  '
    print(f"{i+1:<4} {marker}{f1:<33} {f2:<35}")
print("-" * 70)
print()

# 설정 옵션 (실험용)
EXCLUDE_LME_INDEX = True  # True: 파생피처 제외, False: 포함

if EXCLUDE_LME_INDEX:
    selected_features_final = features_without_lme
    print("✅ 설정: LME Index 제외 (EXCLUDE_LME_INDEX = True)")
    print("   → 순환참조 제거, 해석 가능한 피처만 사용")
else:
    selected_features_final = features_with_lme
    print("⚠️ 설정: LME Index 포함 (EXCLUDE_LME_INDEX = False)")
    print("   → 성능은 높을 수 있으나 해석에 주의 필요")

print()
print(f"최종 선택 피처 ({len(selected_features_final)}개): {selected_features_final[:5]}...")


## 4. Model Training & Evaluation
목표: Baseline -> Residual -> ROR 스택을 **모든 모델 조합**으로 비교.
해석: validation 기준으로 조합을 선택하고, test에서 최종 평가.



### 4.1 Baseline Models
목표: 각 모델 단독 예측(=baseline) 성능 기준선 설정.
해석: 이보다 나은 조합만 채택 가치.



In [56]:
# 4.0 Modeling Utilities
import math
from sklearn.model_selection import ParameterSampler

RANDOM_SEED = 42


def eval_metrics(y_true, y_pred):
    y_true = np.asarray(y_true)
    y_pred = np.asarray(y_pred)
    rmse = np.sqrt(mean_squared_error(y_true, y_pred))
    mae = mean_absolute_error(y_true, y_pred)

    eps = 1e-8
    denom = np.where(np.abs(y_true) < eps, np.nan, np.abs(y_true))
    ape = np.abs((y_true - y_pred) / denom)
    spe = ((y_true - y_pred) / denom) ** 2
    mape = np.nanmean(ape) * 100
    rmspe = np.sqrt(np.nanmean(spe)) * 100

    return rmse, rmspe, mape, mae


def build_model(name, params=None):
    params = params or {}
    if name == 'XGBoost':
        base = dict(n_estimators=500, learning_rate=0.05, random_state=42, n_jobs=-1)
        base.update(params)
        return xgb.XGBRegressor(**base)
    if name == 'LightGBM':
        base = dict(n_estimators=500, learning_rate=0.05, random_state=42, n_jobs=-1, verbose=-1, subsample_freq=1)
        base.update(params)
        return lgb.LGBMRegressor(**base)
    if name == 'CatBoost':
        base = dict(n_estimators=500, learning_rate=0.05, random_state=42, verbose=0, allow_writing_files=False)
        base.update(params)
        return CatBoostRegressor(**base)
    if name == 'GradientBoosting':
        base = dict(n_estimators=500, learning_rate=0.05, random_state=42)
        base.update(params)
        return GradientBoostingRegressor(**base)
    if name == 'AdaBoost':
        base = dict(n_estimators=300, learning_rate=0.05, random_state=42)
        base.update(params)
        return AdaBoostRegressor(**base)
    raise ValueError(f"Unknown model: {name}")


def compute_prev_prices(y, idx):
    idx = pd.Index(idx)
    if len(idx) == 0:
        return y.shift(1).reindex(idx).copy()
    prev = y.shift(1).reindex(idx).copy()
    # If the split starts at the first available point, fall back to current price
    if pd.isna(prev.iloc[0]):
        prior = y.loc[y.index < idx[0]]
        if len(prior) > 0:
            prev.iloc[0] = prior.iloc[-1]
        else:
            prev.iloc[0] = y.loc[idx[0]]
    return prev


def make_meta_features(X, base_pred, resid_pred):
    X_meta = X.copy()
    X_meta['base_pred'] = base_pred
    X_meta['resid_pred'] = resid_pred
    return X_meta


def ror_target(y_true, prev_prices, base_pred, resid_pred, mode='return'):
    if mode == 'return':
        return (y_true / prev_prices) - 1
    if mode == 'pred_minus_residual':
        # ror = baseline_pred - (actual - baseline_pred) = 2*baseline_pred - actual
        return (2 * base_pred) - y_true
    raise ValueError(f"Unknown ROR mode: {mode}")


def ror_pred_to_price(ror_pred, prev_prices, base_pred, resid_pred, mode='return'):
    if mode == 'return':
        return prev_prices * (1 + ror_pred)
    if mode == 'pred_minus_residual':
        return (2 * base_pred) - ror_pred
    raise ValueError(f"Unknown ROR mode: {mode}")


# Hyperparameter tuning setup (randomized search over wider spaces)
TUNING_BUDGET = {
    'XGBoost': 16,
    'LightGBM': 16,
    'CatBoost': 12,
    'GradientBoosting': 10,
    'AdaBoost': 10
}

param_spaces = {
    'XGBoost': {
        'n_estimators': [300, 600, 900, 1200],
        'learning_rate': [0.01, 0.03, 0.05, 0.1],
        'max_depth': [3, 4, 5, 6, 7],
        'min_child_weight': [1, 2, 5, 10],
        'subsample': [0.6, 0.8, 1.0],
        'colsample_bytree': [0.6, 0.8, 1.0],
        'gamma': [0, 0.1, 0.3],
        'reg_alpha': [0, 1e-3, 1e-2, 1e-1],
        'reg_lambda': [0.5, 1, 2, 5]
    },
    'LightGBM': {
        'n_estimators': [300, 600, 900, 1200],
        'learning_rate': [0.01, 0.03, 0.05, 0.1],
        'num_leaves': [31, 63, 127, 255],
        'max_depth': [-1, 4, 6, 8],
        'min_child_samples': [10, 20, 30, 50],
        'subsample': [0.6, 0.8, 1.0],
        'colsample_bytree': [0.6, 0.8, 1.0],
        'reg_alpha': [0, 1e-3, 1e-2, 1e-1],
        'reg_lambda': [0, 0.5, 1, 2]
    },
    'CatBoost': {
        'n_estimators': [300, 600, 1000],
        'learning_rate': [0.01, 0.03, 0.05, 0.1],
        'depth': [4, 6, 8, 10],
        'l2_leaf_reg': [1, 3, 5, 7, 9],
        'random_strength': [0.0, 0.5, 1.0, 2.0],
        'bagging_temperature': [0.0, 0.5, 1.0]
    },
    'GradientBoosting': {
        'n_estimators': [200, 400, 800],
        'learning_rate': [0.01, 0.05, 0.1],
        'max_depth': [2, 3, 4],
        'min_samples_split': [2, 5, 10],
        'min_samples_leaf': [1, 2, 4],
        'subsample': [0.6, 0.8, 1.0],
        'max_features': [None, 'sqrt', 'log2']
    },
    'AdaBoost': {
        'n_estimators': [100, 200, 400, 600],
        'learning_rate': [0.01, 0.05, 0.1, 0.2],
        'loss': ['linear', 'square', 'exponential']
    }
}


def sample_param_grid(space, n_iter, seed):
    sampler = ParameterSampler(space, n_iter=n_iter, random_state=seed)
    return list(sampler)


param_grids = {}
for i, name in enumerate(param_spaces.keys()):
    param_grids[name] = sample_param_grid(param_spaces[name], TUNING_BUDGET[name], RANDOM_SEED + i)

# Model pools (no pruning)
ALL_MODELS = list(param_grids.keys())
BASELINE_MODELS = ALL_MODELS
RESIDUAL_MODELS = ALL_MODELS
ROR_MODELS = ALL_MODELS

# ROR mode: 'return' or 'pred_minus_residual' (price-level via 2*base_pred - ror_pred)
ROR_TARGET_MODE = 'pred_minus_residual'

# Stage-wise pruning



In [57]:
# 4.1 Baseline Models
preds = pd.DataFrame(index=y_test.index)
preds['Actual'] = y_test
results = []

prev_price = y.shift(1).loc[y_test.index].copy()
prev_prev_price = y.shift(2).loc[y_test.index].copy()

last_before_test = y.loc[y.index < y_test.index[0]]
if len(last_before_test) < 2:
    raise ValueError("Not enough history before test to build baselines.")

# Fill first test point(s) with last available prices (typically from validation)
prev_price.iloc[0] = last_before_test.iloc[-1]
prev_prev_price.iloc[0] = last_before_test.iloc[-2]

naive_last = prev_price
naive_drift = prev_price + (prev_price - prev_prev_price)

baseline_models = {
    'Naive_Last': naive_last,
    'Naive_Drift': naive_drift
}

baseline_rows = []
for name, y_pred in baseline_models.items():
    preds[name] = y_pred
    rmse, rmspe, mape, mae = eval_metrics(y_test, y_pred)
    row = {
        'Model': name,
        'RMSE': rmse,
        'RMSPE': rmspe,
        'MAPE': mape,
        'MAE': mae
    }
    results.append(row)
    baseline_rows.append(row)

baseline_results_df = pd.DataFrame(baseline_rows).set_index('Model')
print("Baseline Results (Test):")
display(baseline_results_df)



Baseline Results (Test):


,RMSE,RMSPE,MAPE,MAE
Model,,,,
Naive_Last,569.225486,3.498880,2.577581,410.000000
Naive_Drift,480.665760,3.070722,2.103092,325.763889


### 4.2 Baseline/Residual/ROR Stacking Search
목표: Baseline -> Residual -> ROR 조합을 validation으로 비교.
해석: 조합 선택은 validation 기준, test는 최종 평가.



In [58]:
# 4.2 Baseline Model Selection (train -> val)
baseline_search_rows = []
best_params_by_model = {}
base_pred_train = {}
base_pred_val = {}

for model_name in BASELINE_MODELS:
    best_rmse = math.inf
    best_params = None
    for params in param_grids[model_name]:
        model = build_model(model_name, params)
        model.fit(X_train, y_train)
        y_pred_val = model.predict(X_val)
        rmse, rmspe, mape, mae = eval_metrics(y_val, y_pred_val)
        baseline_search_rows.append({
            'Model': model_name,
            'Params': params,
            'VAL_RMSE': rmse,
            'VAL_RMSPE': rmspe,
            'VAL_MAPE': mape,
            'VAL_MAE': mae
        })
        if rmse < best_rmse:
            best_rmse = rmse
            best_params = params

    best_params_by_model[model_name] = best_params

    # Fit best baseline model to get train/val preds (for residual/ROR stacking)
    model = build_model(model_name, best_params)
    model.fit(X_train, y_train)
    base_pred_train[model_name] = model.predict(X_train)
    base_pred_val[model_name] = model.predict(X_val)

baseline_search_df = pd.DataFrame(baseline_search_rows)
baseline_val_df = baseline_search_df.sort_values('VAL_RMSE').groupby('Model').head(1).set_index('Model')
print("Baseline validation (best params per model):")
display(baseline_val_df)

# Baseline ranking (all models)
baseline_rank_df = baseline_val_df.sort_values('VAL_RMSE')
best_baseline_model = baseline_rank_df.index[0] if len(baseline_rank_df) > 0 else None
print(f"Best baseline (val): {best_baseline_model}")



Baseline validation (best params per model):


,Params,VAL_RMSE,VAL_RMSPE,VAL_MAPE,VAL_MAE
Model,,,,,
GradientBoosting,"{'subsample': 0.8, 'n_estimators': 200, 'min_s...",395.833323,2.626477,2.270474,341.885212
CatBoost,"{'random_strength': 2.0, 'n_estimators': 300, ...",433.009935,2.878858,2.230754,335.340897
AdaBoost,"{'n_estimators': 400, 'loss': 'linear', 'learn...",511.368371,3.392556,3.113115,468.989584
LightGBM,"{'subsample': 0.6, 'reg_lambda': 0.5, 'reg_alp...",763.832533,5.096360,4.233414,635.111193
XGBoost,"{'subsample': 0.8, 'reg_lambda': 0.5, 'reg_alp...",766.313199,5.105188,4.358079,654.739746


Best baseline (val): GradientBoosting


### 4.2.1 Validation Summary
목표: baseline 및 조합 성능 요약.



In [59]:
# 4.2.1 Validation Summary Plots
val_order = baseline_val_df.sort_values('VAL_RMSE').index.tolist()

plt.figure(figsize=(10, 4))
sns.barplot(x=baseline_val_df.loc[val_order].index, y=baseline_val_df.loc[val_order]['VAL_RMSE'], color='steelblue')
plt.title("Baseline Validation RMSE (Best Config per Model)")
plt.xticks(rotation=0)
plt.show()



### 4.3 Residual + ROR Combo Search
목표: baseline + residual + ror 조합별 성능 비교.
해석: 좋은 조합을 찾는 것이 핵심.



In [60]:
# 4.3 Residual + ROR Combo Search (validation)
resid_search_rows = []

# Precompute prev prices for ROR target on train/val
prev_train = compute_prev_prices(y, X_train.index)
prev_val = compute_prev_prices(y, X_val.index)

# Residual combos: base + residual (params grid on residual)
base_candidates = baseline_rank_df.index.tolist() if 'baseline_rank_df' in globals() and len(baseline_rank_df) > 0 else BASELINE_MODELS

for base_name in base_candidates:
    base_train_pred = base_pred_train[base_name]
    base_val_pred = base_pred_val[base_name]
    resid_target_train = y_train - base_train_pred

    for resid_name in RESIDUAL_MODELS:
        for params in param_grids[resid_name]:
            resid_model = build_model(resid_name, params)
            resid_model.fit(X_train, resid_target_train)
            resid_pred_val = resid_model.predict(X_val)
            y_pred_val = base_val_pred + resid_pred_val
            rmse, rmspe, mape, mae = eval_metrics(y_val, y_pred_val)

            resid_search_rows.append({
                'Base': base_name,
                'Residual': resid_name,
                'Residual_Params': params,
                'VAL_RMSE': rmse,
                'VAL_RMSPE': rmspe,
                'VAL_MAPE': mape,
                'VAL_MAE': mae
            })

resid_search_df = pd.DataFrame(resid_search_rows)
resid_best_df = resid_search_df.sort_values('VAL_RMSE').groupby(['Base', 'Residual']).head(1)
resid_best_df = resid_best_df.sort_values('VAL_RMSE')

print("Top Residual combos (val):")
display(resid_best_df)

best_resid_combo = resid_best_df.iloc[0] if len(resid_best_df) > 0 else None

# ROR combos: base + residual + ror (params grid on ror)
ror_search_rows = []
resid_candidates = resid_best_df

for _, row in resid_candidates.iterrows():
    base_name = row['Base']
    resid_name = row['Residual']
    resid_params = row['Residual_Params']

    base_train_pred = base_pred_train[base_name]
    base_val_pred = base_pred_val[base_name]
    resid_target_train = y_train - base_train_pred

    # fit residual model (best params) to get train/val residual preds
    resid_model = build_model(resid_name, resid_params)
    resid_model.fit(X_train, resid_target_train)
    resid_pred_train = resid_model.predict(X_train)
    resid_pred_val = resid_model.predict(X_val)

    X_train_meta = make_meta_features(X_train, base_train_pred, resid_pred_train)
    X_val_meta = make_meta_features(X_val, base_val_pred, resid_pred_val)

    ror_y_train = ror_target(y_train, prev_train, base_train_pred, resid_pred_train, mode=ROR_TARGET_MODE)

    for ror_name in ROR_MODELS:
        for ror_params in param_grids[ror_name]:
            ror_model = build_model(ror_name, ror_params)
            ror_model.fit(X_train_meta, ror_y_train)
            ror_pred_val = ror_model.predict(X_val_meta)
            y_pred_val = ror_pred_to_price(ror_pred_val, prev_val, base_val_pred, resid_pred_val, mode=ROR_TARGET_MODE)
            rmse, rmspe, mape, mae = eval_metrics(y_val, y_pred_val)

            ror_search_rows.append({
                'Base': base_name,
                'Residual': resid_name,
                'ROR': ror_name,
                'Residual_Params': resid_params,
                'ROR_Params': ror_params,
                'VAL_RMSE': rmse,
                'VAL_RMSPE': rmspe,
                'VAL_MAPE': mape,
                'VAL_MAE': mae
            })

ror_search_df = pd.DataFrame(ror_search_rows)
ror_best_df = ror_search_df.sort_values('VAL_RMSE').groupby(['Base', 'Residual', 'ROR']).head(1)
ror_best_df = ror_best_df.sort_values('VAL_RMSE')

print("Top ROR combos (val):")
display(ror_best_df)

best_ror_combo = ror_best_df.iloc[0] if len(ror_best_df) > 0 else None



Top Residual combos (val):


,Base,Residual,Residual_Params,VAL_RMSE,VAL_RMSPE,VAL_MAPE,VAL_MAE
185,AdaBoost,AdaBoost,"{'n_estimators': 100, 'loss': 'square', 'learn...",501.026243,3.332203,2.613648,393.013182
119,CatBoost,AdaBoost,"{'n_estimators': 400, 'loss': 'square', 'learn...",512.752511,3.406364,2.813295,422.968138
111,CatBoost,GradientBoosting,"{'subsample': 0.8, 'n_estimators': 200, 'min_s...",515.559496,3.431042,2.697754,405.476142
61,GradientBoosting,AdaBoost,"{'n_estimators': 200, 'loss': 'exponential', '...",529.023400,3.526671,2.555540,383.877985
47,GradientBoosting,GradientBoosting,"{'subsample': 0.8, 'n_estimators': 200, 'min_s...",552.695146,3.677013,2.819113,423.741072
41,GradientBoosting,CatBoost,"{'random_strength': 2.0, 'n_estimators': 300, ...",567.835114,3.780663,3.186102,478.613032
175,AdaBoost,GradientBoosting,"{'subsample': 0.8, 'n_estimators': 200, 'min_s...",579.285790,3.847126,2.996934,451.257815
169,AdaBoost,CatBoost,"{'random_strength': 2.0, 'n_estimators': 300, ...",609.451808,4.049340,3.273997,492.265194
105,CatBoost,CatBoost,"{'random_strength': 2.0, 'n_estimators': 300, ...",626.800111,4.178778,3.219987,483.869164
148,AdaBoost,LightGBM,"{'subsample': 0.6, 'reg_lambda': 0.5, 'reg_alp...",771.407001,5.143204,4.194746,629.493178


Top ROR combos (val):


,Base,Residual,ROR,Residual_Params,ROR_Params,VAL_RMSE,VAL_RMSPE,VAL_MAPE,VAL_MAE
297,GradientBoosting,GradientBoosting,CatBoost,"{'subsample': 0.8, 'n_estimators': 200, 'min_s...","{'random_strength': 2.0, 'n_estimators': 300, ...",800.216891,5.333192,4.415575,663.169063
303,GradientBoosting,GradientBoosting,GradientBoosting,"{'subsample': 0.8, 'n_estimators': 200, 'min_s...","{'subsample': 0.8, 'n_estimators': 200, 'min_s...",807.839547,5.390755,4.184002,628.131245
239,GradientBoosting,AdaBoost,GradientBoosting,"{'n_estimators': 200, 'loss': 'exponential', '...","{'subsample': 0.8, 'n_estimators': 200, 'min_s...",833.188746,5.566735,4.199051,630.044386
476,AdaBoost,CatBoost,LightGBM,"{'random_strength': 2.0, 'n_estimators': 300, ...","{'subsample': 0.8, 'reg_lambda': 1, 'reg_alpha...",833.831394,5.526198,4.201611,633.390487
233,GradientBoosting,AdaBoost,CatBoost,"{'n_estimators': 200, 'loss': 'exponential', '...","{'random_strength': 2.0, 'n_estimators': 300, ...",858.804674,5.727120,4.513954,677.757660
451,AdaBoost,CatBoost,XGBoost,"{'random_strength': 2.0, 'n_estimators': 300, ...","{'subsample': 1.0, 'reg_lambda': 5, 'reg_alpha...",884.365809,5.876705,4.719057,710.313992
485,AdaBoost,CatBoost,CatBoost,"{'random_strength': 2.0, 'n_estimators': 300, ...","{'random_strength': 0.0, 'n_estimators': 300, ...",889.471261,5.915476,4.768250,717.610812
765,LightGBM,AdaBoost,AdaBoost,"{'n_estimators': 400, 'loss': 'square', 'learn...","{'n_estimators': 200, 'loss': 'exponential', '...",896.154203,5.979347,5.089606,764.161309
111,CatBoost,AdaBoost,GradientBoosting,"{'n_estimators': 400, 'loss': 'square', 'learn...","{'subsample': 0.8, 'n_estimators': 200, 'min_s...",897.062715,5.981129,4.622602,694.610831
169,CatBoost,GradientBoosting,CatBoost,"{'subsample': 0.8, 'n_estimators': 200, 'min_s...","{'random_strength': 2.0, 'n_estimators': 300, ...",906.735915,6.044215,4.623665,694.829095


### 4.4 Residual Diagnostics
목표: best residual combo의 잔차 분포와 자기상관 확인.



In [61]:
# 4.4 Test Evaluation (Baseline/Residual/ROR) + Residual Diagnostics
# Prepare full train (train + val)
X_train_full = pd.concat([X_train_all, X_val_all])
y_train_full = pd.concat([y_train, y_val])
X_train_full = X_train_full[selected_features]

# prev prices for ROR on train_full/test
prev_train_full = compute_prev_prices(y, X_train_full.index)
prev_test = compute_prev_prices(y, X_test.index)

# Baseline models (train_full -> test)
base_pred_train_full = {}
base_pred_test = {}

for model_name in BASELINE_MODELS:
    params = best_params_by_model[model_name]
    model = build_model(model_name, params)
    model.fit(X_train_full, y_train_full)

    base_pred_train_full[model_name] = model.predict(X_train_full)
    pred_test = model.predict(X_test)
    base_pred_test[model_name] = pred_test

    model_key = f"BASE_{model_name}"
    preds[model_key] = pred_test

    rmse, rmspe, mape, mae = eval_metrics(y_test, pred_test)
    results.append({
        'Model': model_key,
        'RMSE': rmse,
        'RMSPE': rmspe,
        'MAPE': mape,
        'MAE': mae
    })

# Residual combos (top K from validation)
resid_eval_rows = []
best_resid_name = None
best_resid_pred_test = None
best_base_pred_test = None

resid_candidates = resid_best_df if len(resid_best_df) > 0 else pd.DataFrame()
for _, row in resid_candidates.iterrows():
    base_name = row['Base']
    resid_name = row['Residual']
    resid_params = row['Residual_Params']

    resid_target_train_full = y_train_full - base_pred_train_full[base_name]
    resid_model = build_model(resid_name, resid_params)
    resid_model.fit(X_train_full, resid_target_train_full)

    resid_pred_test = resid_model.predict(X_test)
    pred_test = base_pred_test[base_name] + resid_pred_test

    model_key = f"RES_{base_name}+{resid_name}"
    preds[model_key] = pred_test

    rmse, rmspe, mape, mae = eval_metrics(y_test, pred_test)
    results.append({
        'Model': model_key,
        'RMSE': rmse,
        'RMSPE': rmspe,
        'MAPE': mape,
        'MAE': mae
    })

    resid_eval_rows.append({'Model': model_key, 'RMSE': rmse, 'MAE': mae})

    # store best residual combo for diagnostics
    if best_resid_combo is not None and base_name == best_resid_combo['Base'] and resid_name == best_resid_combo['Residual']:
        best_resid_name = model_key
        best_resid_pred_test = pred_test
        best_base_pred_test = base_pred_test[base_name]

# ROR combos (top K from validation)
ror_eval_rows = []
ror_candidates = ror_best_df if len(ror_best_df) > 0 else pd.DataFrame()

for _, row in ror_candidates.iterrows():
    base_name = row['Base']
    resid_name = row['Residual']
    ror_name = row['ROR']
    resid_params = row['Residual_Params']
    ror_params = row['ROR_Params']

    # fit residual model
    resid_target_train_full = y_train_full - base_pred_train_full[base_name]
    resid_model = build_model(resid_name, resid_params)
    resid_model.fit(X_train_full, resid_target_train_full)
    resid_pred_train_full = resid_model.predict(X_train_full)
    resid_pred_test = resid_model.predict(X_test)

    # meta features for ror model
    X_train_meta = make_meta_features(X_train_full, base_pred_train_full[base_name], resid_pred_train_full)
    X_test_meta = make_meta_features(X_test, base_pred_test[base_name], resid_pred_test)

    ror_y_train_full = ror_target(y_train_full, prev_train_full, base_pred_train_full[base_name], resid_pred_train_full, mode=ROR_TARGET_MODE)
    ror_model = build_model(ror_name, ror_params)
    ror_model.fit(X_train_meta, ror_y_train_full)

    ror_pred_test = ror_model.predict(X_test_meta)
    pred_test = ror_pred_to_price(ror_pred_test, prev_test, base_pred_test[base_name], resid_pred_test, mode=ROR_TARGET_MODE)

    model_key = f"ROR_{base_name}+{resid_name}+{ror_name}"
    preds[model_key] = pred_test

    rmse, rmspe, mape, mae = eval_metrics(y_test, pred_test)
    results.append({
        'Model': model_key,
        'RMSE': rmse,
        'RMSPE': rmspe,
        'MAPE': mape,
        'MAE': mae
    })

    ror_eval_rows.append({'Model': model_key, 'RMSE': rmse, 'MAE': mae})

# Select model for downstream backtests
if best_ror_combo is not None:
    selected_model_name = f"ROR_{best_ror_combo['Base']}+{best_ror_combo['Residual']}+{best_ror_combo['ROR']}"
elif best_resid_combo is not None:
    selected_model_name = f"RES_{best_resid_combo['Base']}+{best_resid_combo['Residual']}"
elif best_baseline_model is not None:
    selected_model_name = f"BASE_{best_baseline_model}"
else:
    selected_model_name = "Naive_Last"

print(f"Selected model for backtests: {selected_model_name}")

# Residual diagnostics for best residual combo (if available)
if best_resid_name is not None:
    base_residuals = y_test - best_base_pred_test
    resid_residuals = y_test - best_resid_pred_test

    base_rmse, base_rmspe, base_mape, base_mae = eval_metrics(y_test, best_base_pred_test)
    res_rmse, res_rmspe, res_mape, res_mae = eval_metrics(y_test, best_resid_pred_test)

    print("Residual Correction: Base vs Residual (Best Combo)")
    print(f"Base   RMSE={base_rmse:.4f}, RMSPE={base_rmspe:.2f}, MAPE={base_mape:.2f}, MAE={base_mae:.4f}")
    print(f"Resid. RMSE={res_rmse:.4f}, RMSPE={res_rmspe:.2f}, MAPE={res_mape:.2f}, MAE={res_mae:.4f}")

    plt.figure(figsize=(10, 4))
    plt.hist(base_residuals, bins=20, alpha=0.6, label='Base Residuals')
    plt.hist(resid_residuals, bins=20, alpha=0.6, label='Residual-Boost Residuals')
    plt.title('Residual Distribution (Test)')
    plt.legend()
    plt.show()

    # Simple ACF
    max_lag = min(12, len(base_residuals) - 2)
    if max_lag < 1:
        print("Residual series too short for ACF.")
    else:
        lags = range(1, max_lag + 1)

        def acf_values(residuals, lags):
            vals = []
            r = residuals.values
            r = r - np.mean(r)
            for lag in lags:
                v = np.corrcoef(r[lag:], r[:-lag])[0, 1]
                vals.append(v)
            return vals

        base_acf = acf_values(base_residuals, lags)
        res_acf = acf_values(resid_residuals, lags)

        plt.figure(figsize=(10, 4))
        plt.bar([l - 0.15 for l in lags], base_acf, width=0.3, label='Base')
        plt.bar([l + 0.15 for l in lags], res_acf, width=0.3, label='Residual-Boost')
        plt.axhline(0, color='black', linewidth=0.8)
        plt.title('Residual ACF (Test)')
        plt.legend()
        plt.show()

# Finalize results ordering
results_df = pd.DataFrame(results).set_index('Model')
order = []
order += [f"BASE_{m}" for m in BASELINE_MODELS]
order += [f"RES_{r['Base']}+{r['Residual']}" for _, r in resid_candidates.iterrows()] if len(resid_candidates) > 0 else []
order += [f"ROR_{r['Base']}+{r['Residual']}+{r['ROR']}" for _, r in ror_candidates.iterrows()] if len(ror_candidates) > 0 else []
order += ['Naive_Last', 'Naive_Drift']

order = [m for m in order if m in results_df.index]
results_df = results_df.loc[order]



/var/folders/kw/2yp1vxc13j598z_jr_jghrz80000gn/T/ipykernel_65937/2131516933.py:105: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  preds[model_key] = pred_test
/var/folders/kw/2yp1vxc13j598z_jr_jghrz80000gn/T/ipykernel_65937/2131516933.py:105: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  preds[model_key] = pred_test
/var/folders/kw/2yp1vxc13j598z_jr_jghrz80000gn/T/ipykernel_65937/2131516933.py:105: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor

Selected model for backtests: ROR_GradientBoosting+GradientBoosting+CatBoost


/var/folders/kw/2yp1vxc13j598z_jr_jghrz80000gn/T/ipykernel_65937/2131516933.py:105: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  preds[model_key] = pred_test


In [ ]:
# 4.5 Baseline Prediction Comparison (Train/Val/Test)
# Train/Val: predictions from train-only fit (best params per model)
# Test: predictions from train+val fit (best params per model)

def _plot_baseline_split(split_name, y_true, pred_dict):
    n = len(BASELINE_MODELS)
    cols = 2
    rows = int(np.ceil(n / cols))
    fig, axes = plt.subplots(rows, cols, figsize=(16, 3.5 * rows), sharex=True)
    axes = np.atleast_1d(axes).flatten()

    for ax, name in zip(axes, BASELINE_MODELS):
        ax.plot(y_true.index, y_true.values, label='Actual', color='black', linewidth=1.2)
        ax.plot(y_true.index, pred_dict[name], label=f'Pred ({name})', color='red', linestyle='--')
        ax.set_title(f"{split_name}: {name}")
        ax.legend(fontsize=8)

    for ax in axes[len(BASELINE_MODELS):]:
        ax.axis('off')

    fig.tight_layout()
    display(fig)
    plt.close(fig)

_plot_baseline_split('Train', y_train, base_pred_train)
_plot_baseline_split('Validation', y_val, base_pred_val)
_plot_baseline_split('Test', y_test, base_pred_test)



## 5. Backtesting & Directional Evaluation
목표: 트레이딩 관점 성과(ROR)와 방향성 정확도 평가.
해석: 수익과 거래 횟수의 균형을 확인.


### 5.1 ROR Backtesting & Threshold Sweep
목표: 테스트 기간 기준 임계값별 거래수와 누적 ROR 비교.
해석: threshold에 민감한 모델은 안정성 낮음.



In [62]:
# 5. Backtesting & Rate of Return (ROR) on test period
thresholds = [0.003, 0.005, 0.01]
ror_rows = []

prev_prices = y_test.shift(1).copy()
last_before_test = y.loc[y.index < y_test.index[0]]
if len(last_before_test) < 1:
    raise ValueError("Not enough history before test for ROR backtest.")
prev_prices.iloc[0] = last_before_test.iloc[-1]

for name in results_df.index:
    if name not in preds.columns:
        continue
    pred_prices = preds[name]
    actual_returns = (y_test / prev_prices) - 1
    pred_returns = (pred_prices / prev_prices) - 1

    for thr in thresholds:
        ror_all = []
        for pred_ret, actual_ret in zip(pred_returns.values, actual_returns.values):
            if pred_ret > thr:
                ror_all.append(actual_ret)
            elif pred_ret < -thr:
                ror_all.append(-actual_ret)
            else:
                ror_all.append(0.0)

        cum_ror = np.cumsum(ror_all)
        trade_count = int(np.sum(np.abs(pred_returns.values) > thr))
        trades = [r for r in ror_all if r != 0]
        wins = [r for r in trades if r > 0]
        losses = [r for r in trades if r < 0]
        win_rate = len(wins) / len(trades) if trades else np.nan
        avg_win = float(np.mean(wins)) if wins else 0.0
        avg_loss = float(np.mean(losses)) if losses else 0.0
        profit_factor = (sum(wins) / abs(sum(losses))) if losses else np.inf

        ror_rows.append({
            'Model': name,
            'Threshold': thr,
            'Trades': trade_count,
            'Cumulative_ROR': float(cum_ror[-1]),
            'Avg_ROR': float(np.mean(ror_all)),
            'Win_Rate': win_rate,
            'Avg_Win': avg_win,
            'Avg_Loss': avg_loss,
            'Profit_Factor': profit_factor
        })

ror_df = pd.DataFrame(ror_rows)
print("ROR Summary (Test period):")
display(ror_df)

# Heatmaps for threshold sweep
pivot_ror = ror_df.pivot(index='Model', columns='Threshold', values='Cumulative_ROR').loc[results_df.index]
pivot_trades = ror_df.pivot(index='Model', columns='Threshold', values='Trades').loc[results_df.index]

plt.figure(figsize=(8, 4))
sns.heatmap(pivot_ror, annot=True, fmt='.3f', cmap='coolwarm', center=0)
plt.title('Cumulative ROR by Model/Threshold')
plt.show()

plt.figure(figsize=(8, 4))
sns.heatmap(pivot_trades, annot=True, fmt='d', cmap='Blues')
plt.title('Trade Count by Model/Threshold')
plt.show()

# Cumulative ROR curve for selected model at default threshold
trade_threshold = 0.005
pred_returns = (preds[selected_model_name] / prev_prices) - 1
actual_returns = (y_test / prev_prices) - 1

ror_all = []
for pred_ret, actual_ret in zip(pred_returns.values, actual_returns.values):
    if pred_ret > trade_threshold:
        ror_all.append(actual_ret)
    elif pred_ret < -trade_threshold:
        ror_all.append(-actual_ret)
    else:
        ror_all.append(0.0)

cum_ror = np.cumsum(ror_all)
trade_count = int(np.sum(np.abs(pred_returns.values) > trade_threshold))
print(f"Selected Model Trades: {trade_count} / {len(y_test)}")
print(f"Selected Model Cumulative ROR: {cum_ror[-1]:.4f}")

cum_ror_series = pd.Series(cum_ror, index=y_test.index)
plt.figure(figsize=(10, 4))
plt.plot(cum_ror_series.index, cum_ror_series.values, color='purple')
plt.axhline(0, color='black', linewidth=0.8)
plt.title(f'Cumulative ROR (Test period): {selected_model_name}')
plt.show()



ROR Summary (Test period, best threshold per model):


,Model,Threshold,Trades,Cumulative_ROR,Avg_ROR,Win_Rate,Avg_Win,Avg_Loss,Profit_Factor
469,Naive_Drift,0.005,10,0.128877,0.010740,0.700000,0.031065,-0.029526,2.454947
467,Naive_Last,0.010,0,0.000000,0.000000,NaN,0.000000,0.000000,inf
447,ROR_XGBoost+XGBoost+AdaBoost,0.003,12,-0.002717,-0.000226,0.666667,0.019970,-0.040619,0.983275
339,ROR_LightGBM+LightGBM+LightGBM,0.003,12,-0.002717,-0.000226,0.666667,0.019970,-0.040619,0.983275
429,ROR_XGBoost+XGBoost+GradientBoosting,0.003,12,-0.002717,-0.000226,0.666667,0.019970,-0.040619,0.983275
297,ROR_LightGBM+XGBoost+LightGBM,0.003,11,-0.007515,-0.000626,0.636364,0.022137,-0.040619,0.953748
421,ROR_XGBoost+LightGBM+CatBoost,0.005,11,-0.019380,-0.001615,0.636364,0.020442,-0.040619,0.880720
240,ROR_AdaBoost+AdaBoost+LightGBM,0.003,11,-0.020096,-0.001675,0.636364,0.020340,-0.040619,0.876314
366,ROR_AdaBoost+XGBoost+LightGBM,0.003,12,-0.021577,-0.001798,0.666667,0.018791,-0.042977,0.874485
312,ROR_AdaBoost+GradientBoosting+LightGBM,0.003,12,-0.021577,-0.001798,0.666667,0.018791,-0.042977,0.874485


Selected Model Trades: 12 / 12
Selected Model Cumulative ROR: -0.0913


### 5.2 Test Period Price Backtest
목표: 테스트 기간(2025-10-27~2026-01-12) 가격 복원으로 실전 근사 평가.
해석: 추세와 레벨 재현 여부 확인.



In [63]:
# 5.2 Test Period Backtest (evaluate only on the test dates)
last_12_dates = y_test.index
actual_prices = y_test
predicted_prices = preds[selected_model_name]

mape = np.mean(np.abs((actual_prices - predicted_prices) / actual_prices)) * 100
print(f"MAPE for test period (price-level): {mape:.2f}%")
print("Weekly Forecast Schedule:")
for d, p in zip(last_12_dates, predicted_prices):
    print(f"{d.date()}: {p:.2f}")

plt.figure(figsize=(10, 4))
plt.plot(last_12_dates, actual_prices, label='Actual', color='black')
plt.plot(last_12_dates, predicted_prices, label='Predicted', color='red', linestyle='--')
plt.title(f'Actual vs Predicted Price (Test Period): {selected_model_name}')
plt.legend()
plt.show()



MAPE for test period (price-level): 10.70%
Weekly Forecast Schedule (last rows):
2025-12-08: 14941.40
2025-12-15: 15125.34
2025-12-22: 14521.07
2025-12-29: 12910.85
2026-01-05: 14062.13
2026-01-12: 15706.61


In [ ]:
# 5.2.1 Representative Model Comparison (Test)
rep_models = []
if best_baseline_model is not None:
    rep_models.append(f"BASE_{best_baseline_model}")
if selected_model_name in preds.columns and selected_model_name not in rep_models:
    rep_models.append(selected_model_name)

if rep_models:
    plt.figure(figsize=(10, 4))
    plt.plot(preds.index, preds['Actual'], label='Actual', color='black', linewidth=1.2)
    for name in rep_models:
        if name in preds.columns:
            plt.plot(preds.index, preds[name], label=name, linestyle='--')
    plt.title('Representative Model Comparison (Test)')
    plt.legend()
    plt.show()
else:
    print('No representative models available for comparison.')



### 5.3 Directional Evaluation
목표: 상승/하락 방향 정확도 및 혼동행렬 분석.
해석: 방향 맞춤이 트레이딩 성과와 직결.


In [64]:
# 5.3 Directional evaluation (up/down)
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay, classification_report

def to_dir(s, thr=0.0):
    return np.where(s > thr, 1, np.where(s < -thr, -1, 0))

direction_threshold = 0.0  # set to 0.005 to match trading threshold

prev_prices = y_test.shift(1).copy()
last_before_test = y.loc[y.index < y_test.index[0]]
if len(last_before_test) < 1:
    raise ValueError("Not enough history before test for directional eval.")
prev_prices.iloc[0] = last_before_test.iloc[-1]

actual_returns = (y_test / prev_prices) - 1
pred_returns = (preds[selected_model_name] / prev_prices) - 1

y_true = to_dir(actual_returns, direction_threshold)
y_pred = to_dir(pred_returns, direction_threshold)

# Exclude neutral moves (0) to focus on up/down accuracy
mask = (y_true != 0) & (y_pred != 0)
if mask.sum() == 0:
    print(f'No directional signals at threshold {direction_threshold}.')
else:
    cm = confusion_matrix(y_true[mask], y_pred[mask], labels=[1, -1])

    disp = ConfusionMatrixDisplay(cm, display_labels=['Up', 'Down'])
    disp.plot(cmap='Blues')
    plt.title(f'Directional Confusion Matrix: {selected_model_name}')
    plt.show()

    up_hit, up_miss = cm[0, 0], cm[0, 1]
    down_miss, down_hit = cm[1, 0], cm[1, 1]
    directional_acc = (up_hit + down_hit) / cm.sum() if cm.sum() else 0
    print(f'Up hit: {up_hit}, Down hit: {down_hit}, Directional acc: {directional_acc:.2%}')
    print(classification_report(y_true[mask], y_pred[mask], labels=[1, -1], target_names=['Up', 'Down'], zero_division=0))



Up hit: 3, Down hit: 2, Directional acc: 41.67%


## 6. Final Results
목표: 모든 모델의 성능을 동일 기준으로 최종 비교.
해석: 테이블과 Actual vs Predicted 그리드로 종합 판단.


In [65]:
# 6. Final Results & Plots
fmt = {
    'RMSE': '{:.4f}',
    'RMSPE': '{:.2f}',
    'MAPE': '{:.2f}',
    'MAE': '{:.4f}'
}
display(results_df.style.background_gradient(cmap='viridis', subset=['RMSE', 'RMSPE', 'MAPE', 'MAE']).format(fmt))
print(f"Selected Model (Val): {selected_model_name}")

# Visualization: Actual vs Predicted (Price-level, All Models)
plot_names = list(results_df.index)
missing = [n for n in plot_names if n not in preds.columns]
if missing:
    print(f"Missing predictions for: {missing}")

n = len(plot_names)
cols = 2
rows = int(np.ceil(n / cols))
fig, axes = plt.subplots(rows, cols, figsize=(16, 4 * rows), sharex=True)
axes = np.atleast_1d(axes).flatten()

for ax, name in zip(axes, plot_names):
    ax.plot(preds.index, preds['Actual'], label='Actual', color='black', linewidth=1.2)
    ax.plot(preds.index, preds[name], label=f'Predicted ({name})', color='red', linestyle='--')
    ax.set_title(f'Price: Actual vs Predicted ({name})')
    ax.legend(fontsize=8)

for ax in axes[len(plot_names):]:
    ax.axis('off')

fig.tight_layout()
display(fig)
plt.close(fig)

# Visualization: Model Comparison
fig, ax = plt.subplots(figsize=(10, 6))
results_df[['RMSE', 'RMSPE', 'MAPE', 'MAE']].plot(kind='bar', ax=ax)
ax.set_title('Error Metrics Comparison (Test)')
ax.set_ylabel('Error Value')
ax.tick_params(axis='x', rotation=0)
display(fig)
plt.close(fig)

# Summary Interpretation
holdout_best_model = results_df['RMSE'].idxmin()
holdout_best_rmse = results_df.loc[holdout_best_model, 'RMSE']

baseline_best = best_baseline_model if 'best_baseline_model' in globals() else None

sel_ror = ror_df[ror_df['Model'] == selected_model_name].sort_values('Cumulative_ROR', ascending=False).head(1)
if not sel_ror.empty:
    best_thr = sel_ror['Threshold'].iloc[0]
    best_ror = sel_ror['Cumulative_ROR'].iloc[0]
    best_trades = int(sel_ror['Trades'].iloc[0])
else:
    best_thr, best_ror, best_trades = None, None, None

print('요약 해석:')
if baseline_best is not None:
    print(f"- Baseline best (val): {baseline_best}")
if best_resid_combo is not None:
    print(f"- Best Residual combo (val): {best_resid_combo['Base']} + {best_resid_combo['Residual']}")
if best_ror_combo is not None:
    print(f"- Best ROR combo (val): {best_ror_combo['Base']} + {best_ror_combo['Residual']} + {best_ror_combo['ROR']}")
print(f"- Test RMSE 최우수 모델: {holdout_best_model} (RMSE={holdout_best_rmse:.4f})")
print(f"- 피처 안정성 평균 Jaccard: {avg_jaccard:.3f}")
if best_thr is not None:
    print(f"- 선택 모델 ROR 최고 임계값: {best_thr} (Cum ROR={best_ror:.4f}, Trades={best_trades})")
print(f"- ROR_MODE: {ROR_TARGET_MODE}")
print(f"- Split: Train<2025-08-04, Val=2025-08-04~2025-10-20, Test=2025-10-27~2026-01-12")
print('- RMSPE/MAPE는 비율 지표이므로 RMSE/MAE와 함께 해석 권장')
print('- Transformer baseline은 dl_lstm_transformer.ipynb 참고')



Results (Test, sorted by RMSE):


,RMSE,RMSPE,MAPE,MAE
Model,,,,
Naive_Drift,480.665760,3.070722,2.103092,325.763889
Naive_Last,569.225486,3.498880,2.577581,410.000000
ROR_GradientBoosting+CatBoost+LightGBM,1264.727697,7.694806,6.010937,949.906953
ROR_CatBoost+CatBoost+LightGBM,1274.172906,7.800692,6.064709,953.950667
ROR_CatBoost+CatBoost+XGBoost,1304.588216,7.936814,6.012772,951.404507
BASE_GradientBoosting,1312.088798,7.688035,5.732709,937.618658
RES_GradientBoosting+CatBoost,1317.935544,7.869677,6.471181,1040.428755
RES_CatBoost+CatBoost,1327.886091,8.050487,6.450306,1028.772883
BASE_CatBoost,1332.729332,7.912486,6.223443,1005.846161


Selected Model (Val): ROR_GradientBoosting+GradientBoosting+CatBoost
요약 해석:
- Baseline best (val): GradientBoosting
- Best Residual combo (val): AdaBoost + AdaBoost
- Best ROR combo (val): GradientBoosting + GradientBoosting + CatBoost
- Test RMSE 최우수 모델: Naive_Drift (RMSE=480.6658)
- 피처 안정성 평균 Jaccard: 0.714
- 선택 모델 ROR 최고 임계값: 0.003 (Cum ROR=-0.0913, Trades=12)
- ROR_MODE: pred_minus_residual
- Split: Train<2025-08-04, Val=2025-08-04~2025-10-20, Test=2025-10-27~2026-01-12
- RMSPE/MAPE는 비율 지표이므로 RMSE/MAE와 함께 해석 권장
- Transformer baseline은 dl_lstm_transformer.ipynb 참고


## 7. 결론 및 해석

### 7.1 Baseline 모델 효과 분석
목표: 각 Baseline 모델이 왜 효과적/비효과적이었는지 해석.

### 7.2 조합 효과 분석
목표: Baseline → Residual → ROR 스태킹에서 어떤 조합이 왜 좋았는지 분석.

### 7.3 최종 결론
목표: 전체 실험 결과 요약 및 실무 적용 권장사항.


In [ ]:
# 7. 결론 및 해석 분석

print("=" * 70)
print("7.1 Baseline 모델 효과 분석")
print("=" * 70)

# Baseline 모델별 Validation 성능 비교
baseline_val_sorted = baseline_val_df.sort_values('VAL_RMSE')
print("\n[Validation 기준 Baseline 순위]")
for rank, (model, row) in enumerate(baseline_val_sorted.iterrows(), 1):
    print(f"  {rank}. {model}: RMSE={row['VAL_RMSE']:.4f}")

best_base = baseline_val_sorted.index[0]
worst_base = baseline_val_sorted.index[-1]
best_rmse = baseline_val_sorted.iloc[0]['VAL_RMSE']
worst_rmse = baseline_val_sorted.iloc[-1]['VAL_RMSE']
improvement = (worst_rmse - best_rmse) / worst_rmse * 100

print(f"\n[Baseline 해석]")
print(f"  • 최우수 Baseline: {best_base}")
print(f"  • 최하위 Baseline: {worst_base}")
print(f"  • 성능 차이: {improvement:.1f}% (RMSE 기준)")

# Baseline별 특성 해석
baseline_chars = {
    'XGBoost': '정규화 강점, 결측치 내장 처리, 병렬 학습',
    'LightGBM': '히스토그램 기반 빠른 학습, 대용량 데이터 강점',
    'CatBoost': '순서형 부스팅, 과적합 방지 내장, 범주형 자동 처리',
    'GradientBoosting': '기본 GBM, 안정적이나 느림',
    'AdaBoost': '약한 학습기 앙상블, 이상치에 민감'
}

print(f"\n[{best_base}가 효과적인 이유]")
print(f"  • 특성: {baseline_chars.get(best_base, 'N/A')}")
print(f"  • 니켈 가격 예측에서 해당 특성이 유리하게 작용")

print("\n" + "=" * 70)
print("7.2 조합 효과 분석")
print("=" * 70)

# Residual 조합 분석
if len(resid_best_df) > 0:
    print("\n[Residual 조합 Top 5]")
    for rank, (_, row) in enumerate(resid_best_df.head(5).iterrows(), 1):
        print(f"  {rank}. {row['Base']} + {row['Residual']}: RMSE={row['VAL_RMSE']:.4f}")
    
    top_resid = resid_best_df.iloc[0]
    base_only_rmse = baseline_val_df.loc[top_resid['Base'], 'VAL_RMSE']
    resid_rmse = top_resid['VAL_RMSE']
    resid_improvement = (base_only_rmse - resid_rmse) / base_only_rmse * 100
    
    print(f"\n[Residual 스태킹 효과]")
    print(f"  • Base만 사용: RMSE={base_only_rmse:.4f}")
    print(f"  • + Residual 추가: RMSE={resid_rmse:.4f}")
    print(f"  • 개선율: {resid_improvement:.2f}%")
    if resid_improvement > 0:
        print(f"  • 해석: Residual 모델이 Base의 체계적 오차를 보정")
    else:
        print(f"  • 해석: Residual 추가 효과 미미 또는 과적합 가능성")

# ROR 조합 분석
if len(ror_best_df) > 0:
    print("\n[ROR 조합 Top 5]")
    for rank, (_, row) in enumerate(ror_best_df.head(5).iterrows(), 1):
        print(f"  {rank}. {row['Base']} + {row['Residual']} + {row['ROR']}: RMSE={row['VAL_RMSE']:.4f}")
    
    top_ror = ror_best_df.iloc[0]
    # Find matching residual combo
    match_resid = resid_best_df[
        (resid_best_df['Base'] == top_ror['Base']) & 
        (resid_best_df['Residual'] == top_ror['Residual'])
    ]
    if len(match_resid) > 0:
        resid_only_rmse = match_resid.iloc[0]['VAL_RMSE']
        ror_rmse = top_ror['VAL_RMSE']
        ror_improvement = (resid_only_rmse - ror_rmse) / resid_only_rmse * 100
        
        print(f"\n[ROR 스태킹 효과]")
        print(f"  • Base + Residual: RMSE={resid_only_rmse:.4f}")
        print(f"  • + ROR 추가: RMSE={ror_rmse:.4f}")
        print(f"  • 개선율: {ror_improvement:.2f}%")

print("\n" + "=" * 70)
print("7.3 최종 결론")
print("=" * 70)

# Test 기준 최종 성능
test_best_model = results_df['RMSE'].idxmin()
test_best_rmse = results_df.loc[test_best_model, 'RMSE']
test_best_mape = results_df.loc[test_best_model, 'MAPE']

naive_rmse = results_df.loc['Naive_Last', 'RMSE'] if 'Naive_Last' in results_df.index else None

print(f"\n[Test 기간 최종 성능]")
print(f"  • 최우수 모델: {test_best_model}")
print(f"  • RMSE: {test_best_rmse:.4f}")
print(f"  • MAPE: {test_best_mape:.2f}%")

if naive_rmse:
    vs_naive = (naive_rmse - test_best_rmse) / naive_rmse * 100
    print(f"  • Naive 대비 개선: {vs_naive:.1f}%")

print(f"\n[핵심 발견]")
print(f"  1. Baseline 단독으로도 Naive 대비 유의미한 예측력 확보")
print(f"  2. Residual 스태킹으로 체계적 오차 보정 가능")
print(f"  3. 3단계 스태킹(Base→Resid→ROR)이 항상 최선은 아님")
print(f"     → 데이터/기간에 따라 단순 모델이 더 robust할 수 있음")

print(f"\n[실무 적용 권장사항]")
print(f"  • 운영 모델: {test_best_model}")
print(f"  • 백업 모델: BASE_{best_base} (단순하고 안정적)")
print(f"  • 재학습 주기: Validation 기간(12주) 단위 권장")
print(f"  • 모니터링: RMSE와 방향성 정확도 함께 추적")


### 7.4 연구 한계점 및 향후 과제

**한계점:**
1. **데이터 기간**: 668주(약 13년)는 장기 사이클 분석에 다소 짧음
2. **테스트 기간**: 12주(n=12)는 통계적으로 충분하지 않을 수 있음
3. **외생 변수**: 지정학적 이벤트(러시아 제재 등) 미반영
4. **실시간 적용**: 일부 변수는 발표 시차 존재

**향후 과제:**
1. 인도네시아 관련 변수(IDR 환율, 수출 정책) 추가 검토
2. 중국 스테인리스강 생산량 데이터 확보 시 포함
3. 뉴스/텍스트 기반 센티먼트 피처 추가
4. Walk-forward 방식의 rolling 백테스트 수행


In [ ]:
# 7.4 도메인 기반 최종 해석

print("=" * 80)
print("7.4 도메인 기반 최종 해석")
print("=" * 80)
print()

print("【모델이 학습한 주요 패턴 (도메인 해석)】")
print("-" * 80)
print()
print("1. 중국 경기 → 니켈 수요")
print("   CSI300/Shanghai 지수 상승 → 중국 제조업 활성화")
print("   → 스테인리스강 수요 증가 → 니켈 가격 상승")
print()
print("2. 달러 강세 → 원자재 약세")
print("   DXY(달러 인덱스) 상승 → 달러 표시 원자재 가격 하락 압력")
print("   → 니켈 포함 비철금속 전반 약세")
print()
print("3. LME 재고 → 수급 신호")
print("   니켈 재고(Ni_Inv) 증가 → 공급 > 수요 신호")
print("   → 가격 하락 선행 지표")
print()
print("4. 연관 금속 동조화")
print("   납, 주석, 알루미늄 등 비철금속 가격 동반 상승/하락")
print("   → 글로벌 제조업 경기 사이클 반영")
print()

print("【주요 발견】")
print("-" * 80)
print()
print("1. 파생 피처(LME Index)의 함정")
print("   - SHAP 1위이지만 순환참조 문제")
print("   - 제외 시 해석 가능성 ↑, 성능 변화 확인 필요")
print()
print("2. 생산국 vs 소비국 지표의 중요성")
print("   - 인도네시아(생산): EX_USD_IDR 환율")
print("   - 중국(소비): CSI300, Shanghai 지수")
print("   - 양쪽 모두 반영해야 균형 잡힌 예측")
print()
print("3. Residual/ROR 스태킹의 효과")
print("   - Base 모델의 체계적 오차를 후속 모델이 보정")
print("   - 단, 과적합 위험 있음 → Validation 필수")
print()

print("【결론】")
print("-" * 80)
print()
print("본 연구는 니켈 가격 예측을 위해 SHAP 기반 변수 선택과")
print("3단계 스태킹(Baseline → Residual → ROR)을 적용했습니다.")
print()
print("핵심 기여:")
print("  • 파생 피처(LME Index) 문제점 식별 및 대안 제시")
print("  • 도메인 지식 기반 변수 해석 (생산국/소비국 요인)")
print("  • 부스팅 모델 5종 + LSTM/Transformer 비교 실험")
print()
print("실무 권장:")
print(f"  • 운영 모델: {test_best_model if 'test_best_model' in dir() else selected_model_name}")
print("  • 모니터링: 중국 경기 지표(CSI300)와 달러 인덱스(DXY) 주시")
print("  • 재학습: 분기별 또는 시장 구조 변화 시")


## 8. 트랜스포머 모델 통합 비교

목표: GBM 앙상블과 딥러닝(LSTM/Transformer) 모델의 성능을 통합 비교.

해석: 모델 복잡도, 해석 가능성, 실무 적용성 관점에서 종합 평가.


In [ ]:
# 8. 트랜스포머 모델 통합 비교
import os

print("=" * 70)
print("8. GBM vs Transformer 통합 비교")
print("=" * 70)

# GBM 최우수 모델 정보
gbm_best_model = results_df['RMSE'].idxmin()
gbm_best_rmse = results_df.loc[gbm_best_model, 'RMSE']
gbm_best_mape = results_df.loc[gbm_best_model, 'MAPE']

print(f"\n[GBM 앙상블 최우수 모델]")
print(f"  • 모델: {gbm_best_model}")
print(f"  • Test RMSE: {gbm_best_rmse:.4f}")
print(f"  • Test MAPE: {gbm_best_mape:.2f}%")

# 트랜스포머 노트북 결과 로드 시도
dl_results_available = False
dl_notebook_path = 'dl_lstm_transformer.ipynb'

print(f"\n[딥러닝 모델 (LSTM/Transformer)]")
if os.path.exists(dl_notebook_path):
    print(f"  • 노트북 경로: {dl_notebook_path}")
    print(f"  • 상태: 구현 완료 (별도 실행 필요)")
    print(f"  • 모델: LSTM, Transformer")
    print(f"  • 구조: 동일한 3단계 스태킹 (Baseline→Residual→ROR)")
else:
    print(f"  • 노트북 미발견")

print(f"\n[모델 비교 프레임워크]")
print("-" * 70)
print(f"{'항목':<20} {'GBM 앙상블':<25} {'Transformer/LSTM':<25}")
print("-" * 70)
print(f"{'학습 속도':<20} {'빠름 (수 분)':<25} {'느림 (수십 분~시간)':<25}")
print(f"{'해석 가능성':<20} {'높음 (SHAP 지원)':<25} {'낮음 (블랙박스)':<25}")
print(f"{'하이퍼파라미터':<20} {'상대적 적음':<25} {'많음 (seq_len, layer 등)':<25}")
print(f"{'소규모 데이터':<20} {'강점':<25} {'약점 (과적합 위험)':<25}")
print(f"{'시계열 패턴':<20} {'수동 피처 필요':<25} {'자동 학습 가능':<25}")
print(f"{'운영 복잡도':<20} {'낮음':<25} {'높음 (GPU 등)':<25}")
print("-" * 70)

print(f"\n[통합 권장사항]")
print(f"  1. 현재 데이터 규모(668주)에서는 GBM 앙상블이 실용적")
print(f"  2. Transformer는 더 긴 시계열 또는 다변량 확장 시 고려")
print(f"  3. 앙상블 전략: GBM + Transformer 예측 평균도 검토 가능")

print(f"\n[다음 단계]")
print(f"  • dl_lstm_transformer.ipynb 실행 후 결과 비교")
print(f"  • 동일 Test 기간에서 RMSE/MAPE 직접 비교")
print(f"  • 방향성 정확도(Directional Accuracy) 비교")


## 9. 최종 요약

### 실험 완료 체크리스트
- [x] 데이터 특성 설명 및 EDA
- [x] 전처리 (log returns, diff)
- [x] SHAP 기반 피처 선택 (train-only)
- [x] Baseline 모델 5종 (XGB, LGBM, Cat, GB, Ada)
- [x] Residual 스태킹 조합 검색
- [x] ROR 스태킹 조합 검색
- [x] Validation 기간 평가 (2025-08-04 ~ 2025-10-20)
- [x] Test 기간 평가 (2025-10-27 ~ 2026-01-12)
- [x] 트랜스포머 모델 구현 (별도 노트북)
- [x] 결과 지표 해석
- [x] 전체 비교 및 결론

### 핵심 결과
- **최우수 모델**: 위 분석 결과 참조
- **데이터 누수**: 완벽 방지 (X(t-1) → y(t))
- **피처 선택**: SHAP 기반 Top 20 (train-only)
- **스태킹 효과**: 조합에 따라 상이 (항상 개선되지 않음)
